In [1]:
import yaml
import os
import json
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1，准备 一些 电视的图像
# 2，labelme 标注图像 保存为json格式
  # conda install pyqt pillow
  # conda install labelme
  # 在刚才安装好的窗口下输入labelme后便可打开labelme
  # 在label， 选择长方形标注， 标注类型为0， 之后 把一张张图片的标注信息保存到相应的json文件中；
# 3， json文件格式转换为coco的文本文件格式；
  #(py39-torch1.12.1) [lsu@localhost leo_tupian]$ pwd
    #/home/lsu/github/yolov7-main-bamboolsu/leo_tupian
# 4, 转换好的 txt 与 jpg图片放到一起；

# 5， 下面运行 preprocessing.py，  根据leo_training 目录的txt文件，生产imglist，然后放入img_path.txt 文件中 
    #根据img_path.txt的内容， 1:5 的比例拆分为validate.txt 与 train.txt  两个文件 ， 这两个文件里面是jpg的文件列表，用于后面的训练与验证；
    # .


In [2]:
!python3 preprocessing.py

In [ ]:
# 1， 准备数据配置文件 data/leo_cust.yaml 主要内容：
    # 需要修改的地方为5处。第一处：把代码自动下载COCO数据集的命令注释掉，以防代码自动下载数据集占用内存；第二处：修改train的位置为train_list.txt的路径；第三处：修改val的位置为val_list.txt的路径；第四处：修改nc为数据集目标总数；第五处：修改names为数据集所有目标的名称。然后保存。
    # 原文链接：https://blog.csdn.net/qq_29788741/article/details/126678648
    #train: train.txt  # 118287 images
    #val: validate.txt  # 5000 images

    ## number of classes
    #nc: 1

    ## class names leo add
    #names: ['tv']

# 2, 准备模型配置文件   cfg/traning/leo_yolov7.yaml 主要修改训练类别的个数
    ## parameters
    #nc: 1  # number of classes
    
# 3， 运行 训练； 训练用到之前提到的三个文件：预训练模型yolov7_training.pt、yolov7-Helmet.yaml和Helmet.yaml，当然还有一些其他的超参数
    # python train.py --weights weights/yolov7.pt --cfg cfg/training/yolov7-Helmet.yaml --data data/Helmet.yaml --device 0,1 --batch-size 64 --epoch 10
    #  原文链接：https://blog.csdn.net/qq_29788741/article/details/126678648
    # 然后运行界面如下，图中红色部分表示开始扫描数据，然后会在yolov7/datasets/Helmet/下生成两个缓存文件，这两个文件是方便模型进行数据读取生成的，训练过程中不要删除。扫描完后模型会开始训练，如下图


# 4， 训练结果 ； 训练结束后，终端会打印出最好的模型和最后一个epoch的模型结果保存在哪里
    # 在/runs/train/exp/下，也存了其他的训练结果文件，比如每个epoch的结果到保存到了.txt文件夹里
    

In [ ]:
!python3 train.py --data data/cust.yaml --cfg cfg/training/yolov7-tiny.yaml --hyp data/hyp.scratch.custom.yaml --device 0,1 --img-size 832 --weights runs/train/kk2/weights/last.pt --name lsu33 --sync-bn --batch-size 16 --epochs 50

In [ ]:
!python3 yolov7_train_pruned.py --data data/cust.yaml --cfg cfg/deploy/yolov7-tiny.yaml --hyp data/hyp.scratch.custom.yaml --device 0,1 --img-size 832 --weights yolov7-tiny.pt --name lsu1 --sync-bn --batch-size 16 --epochs 999

120.0

In [ ]:
#import torch
#print(torch.__version__)   #1.12.1+cu116
!python train.py --data data/custom_kpts.yaml --cfg cfg/yolov7-w6-pose_custom.yaml --hyp data/hyp.pose.yaml --device 0 --kpt-label --epochs 600

# Test:

In [ ]:
#!python test.py --data data/custom_kpts.yaml --img 640 --conf 0.001 --iou 0.65 --weights runs/train/exp/weights/best.pt --kpt-label

# Detect:

In [ ]:
!python3 yolov7_detect_pruned.py --weights runs/train/ycx16/weights/best.pt --source taiyuan_cx_ycx --conf 0.45 --iou 0.10

In [ ]:
# 预测/推断， /推理
    #official detect
    #python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source inference/images/20231207103708.jpg 

    #leo detect --conf 0.45 --iou 0.10
    #python  detect.py --weights ./runs/train/yolov7-epoch100-batch3-worker6/weights/best.pt --source  ./test/ 


# 预测、推断结果查看
    # 位置： runs/train/yolov7-epoch150-batch4-worker 里面是整个结果
        #此目录下的weights是权重；

In [ ]:
!python3.6 detect.py --weights ../weights/hd/weights/best.pt --source test --conf 0.45 --iou 0.10

In [ ]:
# 训练结果分析  ; 结果在 run/train/yolov7-epoch300-batch32-worker6 目录下面

# 1， 权重 weight ;  在 run/train/yolov7-epoch300-batch32-worker6/weight目录下面； 
    # init.pt, epoch_000.pt, epoch_024.pt, epoch_xxx.pt, epoch_099.pt, last.pt, best.pt

# 2，召回率与精确率； confusion_matrix.png ； 在run/train/yolov7-epoch300-batch32-worker6/ 目录下面；
    # TP（True Positive）: 将正类预测为正类数 即正确预测，真实为0，预测也为0
    # FN （False Negative）：将正类预测为负类 即错误预测，真实为0，预测为1
    # FP（False Positive）：将负类预测为正类数 即错误预测， 真实为1，预测为0
    # TN （True Negative）：将负类预测为负类数，即正确预测，真实为1，预测也为1

    #精确率和召回率的计算方法
        # 精确率Precision=TP / (TP+FP), 在预测是Positive所有结果中，预测正确的比重
        # 召回率recall=TP / (TP+FN), 在真实值为Positive的所有结果中，预测正确的比重
    #原文链接：https://blog.csdn.net/weixin_43397302/article/details/127073824


# 3， F1分数， 它被定义为查准率和召回率的调和平均数；  F1_curve.png  一些多分类问题的机器学习竞赛，常常将F1-score作为最终测评的方法。它是精确率和召回率的调和平均数


# 4， hyp.yaml和opt.yaml；  训练时的超参数以及train.py中间的参数

# 5， P_curve.png； 准确率precision和置信度confidence的关系图
    # 通过访问模型的conf_thresh属性获取当前模型的置信阈值，并将其打印出来； 
    # 获取当前模型的置信阈值
    # conf_thresh = model.conf_thresh
    # print(f"当前模型的置信阈值为：{conf_thresh}")

# 6， PR_curve.png ； PR曲线中的P代表的是precision（精准率），R代表的是recall（召回率），其代表的是精准率与召回率的关系，一般情况下，将recall设置为横坐标，precision设置为纵坐标。
    #   PR曲线下围成的面积即AP，所有类别AP平均值即Map.
    # Precision和Recall往往是一对矛盾的性能度量指标；及一个的值越高另一个就低一点；
        #提高Precision <==> 提高二分类器预测正例门槛 <==> 使得二分类器预测的正例尽可能是真实正例；
        #提高Recall <==> 降低二分类器预测正例门槛 <== >使得二分类器尽可能将真实的正例挑选
        #原文链接：https://blog.csdn.net/weixin_43397302/article/details/127073824

# 7， R_curve.png； 召回率recall和置信度confidence之间的关系

# 8 ，results.png；
    # Box：Box推测为GIoU损失函数均值，越小方框越准；
    # Objectness：推测为目标检测loss均值，越小目标检测越准；
    #   #Classification：推测为分类loss均值，越小分类越准，本实验为一类所以为0；
    # Precision：精度（找对的正类/所有找到的正类）；
    # Recall：真实为positive的准确率，即正样本有多少被找出来了（召回了多少）。
        # Recall从真实结果角度出发，描述了测试集中的真实正例有多少被二分类器挑选了出来，即真实的正例有多少被该二分类器召回。
    # val BOX:  验证集bounding box损失
    # val Objectness：验证集目标检测loss均值
    # val classification：验证集分类loss均值，本实验为一类所以为0
    # mAP是用Precision和Recall作为两轴作图后围成的面积，m表示平均，@后面的数表示判定iou为正负样本的阈值，@0.5:0.95表示阈值取0.5:0.05:0.95后取均值。
    # mAP@.5:.95（mAP@[.5:.95]）
        #表示在不同IoU阈值（从0.5到0.95，步长0.05）（0.5、0.55、0.6、0.65、0.7、0.75、0.8、0.85、0.9、0.95）上的平均mAP。
    # @mAP@.5：表示阈值大于0.5的平均mAP
    #  一般训练结果主要观察精度和召回率波动情况（波动不是很大则训练效果较好）
        # 然后观察mAP@0.5 & mAP@0.5:0.95 评价训练结果。
    # 原文链接：https://blog.csdn.net/weixin_43397302/article/details/127073824

# 9, results.txt
    # 分别的含义是训练次数、GPU消耗、训练集边界框损失、训练集目标检测损失、训练集分类损失[一般是 0 ]、训练集总损失、
    # targets目标【目标的个数，一般是正整数】、输入图片大小、
    # Precision、Recall、mAP@.5、mAP@.5:.95、
    # 验证集边界框损失、验证集目标检测损失、验证机分类损失【一般是 0】

# 10，train_batch1.jpg ,   train_batchx, 设置的batch_size为8所以一次读取8张图片
# 11, test_batch0_labels.jpg;   test_batchx_labels ;验证集第一轮的实际标签


# 12, 运行tensorboard
    # activate yolov7(自己所配的环境名称)
    # tensorboard --logdir=训练结果所在的文件夹
